In [1]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 54.3MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 139kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 1.4MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 33.2MB/s 
     |████████████████████████████████| 184kB 12.0MB/s 
     |████████████████████████████████| 235kB 13.4MB/s 
     |████████████████████████████████| 2.2MB 29.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.1-cp36-none-any.whl size=308552 sha256=acb64a87a3b3579ba4a763907a8fa73fd94d5a6da3095f6f992e2cb2f038b274
  Stored in directory: /root/.cache/pip/wheels/e6/25/ea/3d71d2088dccc63214fa59259dcc598ded4150a5f8b41d84ff


In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv,GlobalAttention
from torch.nn.functional import one_hot
from torch_geometric.datasets import QM9
from torch.nn.parameter import Parameter
from torch_geometric.data import DataLoader
import torch.nn as nn
from torch.nn import CosineSimilarity
from matplotlib import pyplot as plt

In [3]:
data_QM9 = QM9('/content/data')

Extracting /content/data/raw/qm9_v2.zip
Processing...
Using a pre-processed version of the dataset. Please install `rdkit` to alternatively process the raw data.
Done!


data = dataset[0] get one sample https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html
 

In [30]:
g = data_QM9[100000]
data = DataLoader(data_QM9,10,True)

In [5]:
for g in data_QM9:
  if g.edge_attr.size()[1] != 4:
    print('bug')

In [6]:
for g in data_QM9:
  if g.x.size()[1] != 11:
    print('bug')

In [7]:
def A_mat(data):   ### torch geometric graph to adjecency tensor

  edge_type  = data.edge_attr
  n_nodes    = data.num_nodes
  adj_mat    = torch.zeros((n_nodes,n_nodes,edge_type.size()[1]))
  edge_index = data.edge_index
  
  for i in range(data.num_edges):
    k, m = edge_index[0,i],edge_index[1,i]
    adj_mat[k,m,:] = edge_type[i,:]

  return adj_mat.to(dtype = torch.float32)

'''
    R_i∈{0,…,|R|−1}
'''

def edge_type(data):
  return torch.matmul(data.edge_attr, torch.tensor([0.,1.,2.,3.]).T).to(torch.long)


def batch_convolve(batch,Conv_CUSTM):  #### 
  return torch.cat([Conv_CUSTM(A_mat(m).unsqueeze(0), m.x.unsqueeze(0)) for m in batch.to_data_list()],dim=1)

### The following convolution tested 

In [8]:
class Convolve(torch.nn.Module):
    def __init__(self,in_channels,out_channels,n_relations):
      super(Convolve,self).__init__()

      '''
       unclear type of weight initialization 
      '''

      self.weight     = Parameter(torch.nn.init.xavier_uniform_(torch.empty(in_channels,out_channels,n_relations), gain=1.0))
      self.lin        = nn.Linear(in_channels,out_channels)
      
    def forward(self,A,x):

      #A       = A[:,:,:,:-1]
      sum_     = torch.sum(A,dim=2)
      norm     = 1./(sum_ + torch.full(sum_.size(),1e-15))  ## look for analytical solution
      A_new    = torch.einsum('sijcd,sicd->sijcd',A.unsqueeze(4),norm.unsqueeze(3))
      Theta_ij = torch.einsum('abc,sijcd->sijabd',self.weight,A_new).squeeze(-1)
      x_new    = torch.einsum('sja,sijab->sib',x,Theta_ij) + self.lin(x)

      return x_new

### Convolve check 

### Initialise custom and libriary convolution 
### operators with the same weights 


In [9]:
Conv_RGCN  = RGCNConv(11,3,4,root_weight=True)
Conv_CUSTM = Convolve(11,3,4)

params = []
for param in Conv_RGCN.parameters():
  params.append(param.data)

theta, theta_root,_ = params

Conv_CUSTM.weight.data = theta.permute(1,2,0)
Conv_CUSTM.lin.weight.data = theta_root.T

#Conv_CUSTM.lin.bias.data    = torch.tensor([1.,2.,0.3])
#Conv_RGCN.bias.data    = torch.tensor([1.,2.,0.3])

Conv_CUSTM.lin.bias.data  = Conv_RGCN.bias.data

### Compute node features for all graphs in QM9 dataset 

### Convolve check

In [10]:
cos = nn.CosineSimilarity(dim=0)
v1_ = []
v2_ = []
simillarity = []
for batch in data:
  v1  = batch_convolve(batch,Conv_CUSTM)[0].flatten()
  v2  = Conv_RGCN(batch.x,batch.edge_index,edge_type(batch)).flatten()
  sim = cos(v1,v2).detach().numpy()
  if sim !=1:
    simillarity.append(sim)
    v1_.append(v1)
    v2_.append(v2)

In [11]:
print('proportion of batches which do not have simmilarity equal to one = {}'.format(len(simillarity)/len(data)))
print('divergence upper bound ={},divergence minimum bound ={}'.format(max(simillarity),min(simillarity)))

proportion of batches which do not have simmilarity equal to one = 0.11173952919596454
divergence upper bound =1.000000238418579,divergence minimum bound =0.9999998211860657


In [12]:
v1_[simillarity.index(max(simillarity))][:5], v2_[simillarity.index(max(simillarity))][:5]

(tensor([ 0.7653,  2.5251,  1.1903, -6.3267,  5.5478], grad_fn=<SliceBackward>),
 tensor([ 0.7653,  2.5251,  1.1903, -6.3267,  5.5478], grad_fn=<SliceBackward>))

In [13]:
v1_[simillarity.index(min(simillarity))][:5],v2_[simillarity.index(min(simillarity))][:5]

(tensor([ 1.5972,  1.5825,  1.8130, -5.0916,  6.6822], grad_fn=<SliceBackward>),
 tensor([ 1.5972,  1.5825,  1.8130, -5.0916,  6.6822], grad_fn=<SliceBackward>))

### Convolve-Aggregate-Check 

In [68]:
hgate   = nn.Linear(3,1)
h_theta = nn.Linear(3,2)
lin_last = nn.Linear(2,1)
'''
  batch is a column vector which maps each node to its respective graph ###in the batch:
  batch=[0⋯01⋯n−2n−1⋯n−1]⊤
'''

'\n  batch is a column vector which maps each node to its respective graph ###in the batch:\n  batch=[0⋯01⋯n−2n−1⋯n−1]⊤\n'

In [92]:
class Net1(nn.Module):
  def __init__(self,conv,gate_nn,nn,lin_last):
    super(Net1, self).__init__()
    self.agg  = GlobalAttention(gate_nn, nn)
    self.conv = conv
    self.lin  = lin_last
  def forward(self,x):
    x_new = self.conv(x.x,x.edge_index,edge_type(x))
    output = self.agg(x_new,x.batch)
    return self.lin(output)

class Net2(Net1):
  def forward(self,x):
    x_new = torch.cat([self.conv(A_mat(m).unsqueeze(0), m.x.unsqueeze(0)) for m in x.to_data_list()],dim=1)
    output = self.agg(x_new.squeeze(0),x.batch)
    return self.lin(output)

In [93]:
N1 = Net1(Conv_RGCN,hgate,h_theta,lin_last)
N2 = Net2(Conv_CUSTM,hgate,h_theta,lin_last)

In [95]:
v1_ = []
v2_ = []
simillarity = []
for batch in data:
  v1  = N1(batch).flatten()
  v2  = N2(batch).flatten()
  sim = cos(v1,v2).detach().numpy()
  if sim !=1:
    simillarity.append(sim)
    v1_.append(v1)
    v2_.append(v2)

In [96]:
print('proportion of batches which do not have simmilarity equal to one = {}'.format(len(simillarity)/len(data)))
print('divergence upper bound ={},divergence minimum bound ={}'.format(max(simillarity),min(simillarity)))

proportion of batches which do not have simmilarity equal to one = 0.22623051054723325
divergence upper bound =1.0000001192092896,divergence minimum bound =0.9999997615814209


### Generator Check 

In [ ]:
def permute3D(A):

  A = A.permute(2,0,1)
  A_new = torch.triu(A,diagonal=1) + torch.transpose(torch.triu(A,diagonal=1),1,2)

  return A_new.permute(1,2,0)

def permute4D(A):

  bz,n,m,k = A.size()

  A_new = A.permute(1,2,3,0)
  A_new = A_new.reshape(n,m,-1)
  A_new = permute3D(A_new)
  A_new = A_new.reshape(n,m,k,bz)
  A_new = A_new.permute(3,0,1,2)

  return A_new


class Generator(torch.nn.Module):

    # N - maximum number of atoms
    # T - number of atom types
    # Y - number of bond types

    #### Candidates to enforce A_sym = LL^T

    def __init__(self,z_dim,N,T,Y,temp):
      super(Generator, self).__init__()

      self.N = N
      self.T = T
      self.Y = Y

      self.temp = temp ### GambelSoftmax activation - temperature

      self.lin1 = nn.Linear(z_dim,128)
      self.lin2 = nn.Linear(128,256)
      self.lin3 = nn.Linear(256,512)

      self.edges = nn.Linear(512,self.N*self.N*self.Y)
      self.nodes = nn.Linear(512,self.N*self.T)

      self.act   = nn.functional.gumbel_softmax

    def forward(self,x):

        output      = self.lin3(self.lin2(self.lin1(x)))

        nodes_logit = self.nodes(output).view(-1,self.N,self.T)
        nodes       = self.act(nodes_logit,dim=-1,tau=self.temp,hard=True)

        edges_logit      = self.edges(output).view(-1,self.N,self.N,self.Y)
        edges_logit_T    = torch.transpose(edges_logit,1,2)
        edges_logit      = 0.5*(edges_logit+edges_logit_T)

        edges_logit      = self.act(edges_logit,dim=-1,tau=self.temp,hard=True)

        edges = permute4D(edges_logit)

        return  nodes, edges

In [ ]:
G = Generator(10,9,5,4,0.1)

def tensor_symmetry_check(A):
  return torch.sum(A-torch.transpose(A,0,1)).detach().numpy()

for _ in range(1000):
  z = torch.rand((60,10))
  X,A = G(z)
  for a in A:
    if tensor_symmetry_check(a) !=0 or torch.sum(torch.diagonal(a,dim1=0,dim2=1)).detach().numpy() !=0:
      print('not symmetrical tensor or diagonal does not equal to [0,0,0,0]') 

In [ ]:
torch.diagonal(a,dim1=0,dim2=1).reshape(9,4)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], grad_fn=<UnsafeViewBackward>)